In [78]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [79]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
print("Training data dimensions =", train.shape)
print("Testing data dimensions =", test.shape)

Training data dimensions = (2000, 5)
Testing data dimensions = (200, 4)


In [80]:
train.head()

,T,V,P,RH,E
0,273.39,420.12,91.84,57.41,1778.12
1,195.26,248.88,92.29,35.21,1824.16
2,377.52,360.42,92.31,27.69,1761.72
3,171.60,244.98,91.61,56.99,1889.64
4,222.56,353.70,92.43,54.25,1806.68


In [81]:
y_train = train.pop('E')

In [82]:
'''fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(7, 7))
sns.distplot(train['T'], color='r', ax=ax[0][0])
sns.distplot(train['V'], color='b', ax=ax[0][1])
sns.distplot(train['P'], color='g', ax=ax[1][0])
sns.distplot(train['RH'], ax=ax[1][1])
plt.tight_layout()'''

"fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(7, 7))\nsns.distplot(train['T'], color='r', ax=ax[0][0])\nsns.distplot(train['V'], color='b', ax=ax[0][1])\nsns.distplot(train['P'], color='g', ax=ax[1][0])\nsns.distplot(train['RH'], ax=ax[1][1])\nplt.tight_layout()"

In [83]:
train_rows = train.shape[0]
data = pd.concat([train, test])

del train
del test

# Preprocessing

In [84]:
#data.drop(['RH'], inplace=True, axis=1)
#data['T'] = data['T'] + 273

In [85]:
data.head()

,T,V,P,RH
0,273.39,420.12,91.84,57.41
1,195.26,248.88,92.29,35.21
2,377.52,360.42,92.31,27.69
3,171.60,244.98,91.61,56.99
4,222.56,353.70,92.43,54.25


In [86]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
for c in data.columns:
    data[c] = sc.fit_transform(data[[c]])    

In [87]:
data = data.values
train = data[:train_rows]
test = data[train_rows:]

del data

In [88]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [89]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=25, verbose=0)

In [90]:
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -2357947.81 (158256.47) MSE


In [91]:
estimator.fit(train, y_train)
predictions = estimator.predict(test)

In [92]:
predictions = pd.DataFrame({'E': predictions})

In [93]:
predictions

,E
0,208.579071
1,186.741852
2,309.921997
3,322.270874
4,181.368225
5,247.764008
6,353.645630
7,175.867920
8,637.383789
9,234.219574


In [94]:
predictions.to_csv('NN_submission.csv', index=False, header=False)